In [2]:
!pip install langfuse openai --upgrade


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
from langfuse.openai import openai

completion = openai.ChatCompletion.create(
  name="test-chat-openai",
  model="gpt-3.5-turbo",
  messages=[
      {"role": "system", "content": "You are a very accurate calculator. You output only the result of the calculation."},
      {"role": "user", "content": "1 + 1 = "}],
  temperature=0,
  metadata={"someMetadataKey": "someValue"},
)

In [5]:
!pip install pydantic==1.* --upgrade

zsh:1: no matches found: pydantic==1.*


In [6]:
from typing import List
from pydantic import BaseModel
 
class StepByStepAIResponse(BaseModel):
    title: str
    steps: List[str]
schema = StepByStepAIResponse.schema() # returns a dict like JSON schema

In [7]:
import json
response = openai.ChatCompletion.create(
    name="test-function",
    model="gpt-3.5-turbo-0613",
    messages=[
       {"role": "user", "content": "Explain how to assemble a PC"}
    ],
    functions=[
        {
          "name": "get_answer_for_user_query",
          "description": "Get user answer in series of steps",
          "parameters": StepByStepAIResponse.schema()
        }
    ],
    function_call={"name": "get_answer_for_user_query"}
)
 
output = json.loads(response.choices[0]["message"]["function_call"]["arguments"])

In [8]:
from uuid import uuid4
trace_id = str(uuid4())

In [9]:
country = openai.ChatCompletion.create(
  name="random-country",
  model="gpt-3.5-turbo",
  messages=[
      {"role": "user", "content": "Pick a random country"}],
  temperature=1,
  trace_id=trace_id
)["choices"][0]["message"]["content"]
 
capital = openai.ChatCompletion.create(
  name="geography-teacher",
  model="gpt-3.5-turbo",
  messages=[
      {"role": "system", "content": "You are a Geography teacher helping students learn the capitals of countries. Output only the capital when being asked."},
      {"role": "user", "content": country}],
  temperature=0,
  trace_id=trace_id
)["choices"][0]["message"]["content"]
 
poem = openai.ChatCompletion.create(
  name="poet",
  model="gpt-3.5-turbo",
  messages=[
      {"role": "system", "content": "You are a poet. Create a poem about a city."},
      {"role": "user", "content": capital}],
  temperature=1,
  max_tokens=200,
  trace_id=trace_id
)["choices"][0]["message"]["content"]

In [10]:
from langfuse import Langfuse
from langfuse.model import InitialScore
 
langfuse = Langfuse()
 
langfuse.score(InitialScore(
    traceId=trace_id,
    name="my-score-name",
    value=1
));